![Image alt](https://github.com/zoLikeCode/system_hack_mts/raw/main/mtstruetech.png)

---
### Содержание / навигация:
1. [Описание](#описание)  
2. [Установка и запуск проекта](#установка-и-запуск-проекта)  
3. [Основной функционал проекта](#основной-функционал-проекта)  
4. [Технологии и инструменты](#технологии-и-инструменты)  
5. [Команда проекта](#команда-проекта)  
6. [Архитектура проекта](#архитектура-и-структура-проекта)  
7. [Демонстрация работы проекта](#демонстрация-работы-проекта)  
8. [Заключение](#заключение)  

---

### Описание:
Наша команда разработала проект, который помогает создать инклюзивную среду в многоквартирных домах с использованием технологий умного дома. Мы ориентируемся на потребности маломобильных и слабослышащих людей, предлагая инновационные решения для улучшения их повседневной жизни и повышения безопасности. Наше приложение позволяет автоматизировать ключевые элементы инфраструктуры — пандусы, двери, экстренные оповещения — улучшает взаимодействие с домофонами и облегчает навигацию для людей с ограниченными возможностями. Это помогает создать более удобные и безопасные условия для всех жильцов.


---

### Установка и запуск проекта

**Back:**
1. Перейдите на сайт [https://console.mistral.ai](https://console.mistral.ai) и получите свой API ключ.
2. В папке [`ML/NLP/`](ML/NLP/) создайте файл `.env` на основе примера [`.env.example`](ML/NLP/.env.example)
3. После настройки `.env` файла выполните команду для сборки и запуска проекта:

   ```bash
   docker-compose up --build
---

### Основной функционал проекта:

1. ИИ-ассистент, заточенный под ЖК.
2. Bluetooth-метки для автоматизации инклюзивных элементов
3. Автоматические оповещения о чрезвычайных ситуациях
4. Расшифровка речи для граждан с нарушениями слуха
5. Функция "Кто дома?"
6. Персонализированнный сценарий для членов семьи

---

### Технологии и инструменты:

1. ml: Vosk(Speech2Text), LangChain(LLM).
2. backend: LangServe, FastAPI, unicorn.
3. mobile: React Native, Expo Router, Expo SDK, Expo GO, Jotai, AsyncStorage.

--- 

### Команда проекта:

1. Светлана Шубина - продакт-менеджер, дизайнер.
2. [Никита Зонтов](https://github.com/zoLikeCode) - backend-разработчик.
3. [Денис Басанский](https://github.com/Bigilittle) - ml-специалист.
4. [Павел Шабуров](https://github.com/Shavelo) - ml-специалист.
5. [Максим Меркулов](https://github.com/spioncino) - mobile-разработчик.

---

### Архитектура проекта:

---

### Демонстрация работы проекта:

1. [Видео]()
2. [Презентация]()



In [10]:
API_KEY = "aj3ozp7g0tZaWyvbmP8pHPrNGRgfJN7d"

In [11]:

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(api_key = API_KEY, model="mistral-large-latest")

In [12]:
llm.invoke('привет, как дела?')

AIMessage(content='Привет! У меня всё хорошо, спасибо. А у тебя как дела?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 10, 'total_tokens': 38, 'completion_tokens': 28}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-9019417b-7888-42e9-8826-04615caf76dd-0', usage_metadata={'input_tokens': 10, 'output_tokens': 28, 'total_tokens': 38})

In [13]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.tools import tool
from pydantic import Field
from langchain.chains import LLMChain
import time
from langchain_core.messages import HumanMessage, SystemMessage


class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    age: int



people_status = {
    "1001": {"status": "свободен"},
    "1002": {"status": "занят"},
    "1003": {"status": "свободен"},
    "1004": {"status": "занят"},
    "1005": {"status": "свободен"}
}




graph_builder = StateGraph(State)
@tool
def get_order_status(employee_id: str = Field(description="Identifier of Employee")) -> str:
    """give the employee status. use when you need get a status of employee"""
    return people_status.get(employee_id, f"Не существует сотрудника")


tools = [get_order_status]
llm = ChatMistralAI(api_key = API_KEY, model="mistral-large-latest", temperature=0)
llm_with_tools = llm.bind_tools(tools)


message = [
    ("system", "Отвечай только на русском языке. Ты являешься **ДОМОФОНОМ** и всегда общаешься уважительно и очень коротко, и отвечаешь только на вопросы человека про себя, или тот функцианал что у тебя есть"),
    ("system", "Человека зовут **{name}** и ему **{age}**"),
    ("human", "{message}")
]
promt = ChatPromptTemplate(message)


chain = promt | llm_with_tools




def chatbot(state: State) -> State:
    """Обрабатывает сообщения и обновляет состояние."""
    time.sleep(1)

    res = chain.invoke({
        "message": state['messages'],
        "name": state['name'],
        "age": state['age']
    })

    if "tool_calls" in res.additional_kwargs:
        return {
            "messages": res,
            "name": state['name'],
            "age": state['age']
        }

    return {
        "messages": {"role": "ai", "content": res.content},
        "name": state['name'],
        "age": state['age']
    }






graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[get_order_status])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [14]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [15]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.tools import tool
from pydantic import Field, BaseModel
import time
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os
import json



load_dotenv('../.env')
API_KEY = os.getenv('API_KEY')

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    age: int


graph_builder = StateGraph(State)


def read_json_file(file_path: str) -> dict:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


def write_json_file(file_path: str, data: dict) -> None:
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)


status_pass = read_json_file("./json/status_pass.json")
plumbers_requests = read_json_file('./json/plumbers_requests.json')

class Employee(BaseModel):
    full_name: str = Field(description="full name of the person who needs to be issued a pass")
    tenant_name: str = Field(description="The name of the person who orders the service")

class ApartmentRequest(BaseModel):
    apartment_number: str = Field(description="the apartment number where the service is ordered")
    resident_name: str = Field(description="name of the customer of the service")
    time_of_visit: str = Field(description="time to visit")



@tool(args_schema=Employee)
def add_employee_pass(full_name: str, tenant_name: str) -> str:
    """It is used when it is necessary to provide a pass to a person, it is recorded in the name of the customer."""
    employee_id = full_name.lower().replace(" ", "_")  
    status_pass[employee_id] = tenant_name

    write_json_file("./json/status_pass.json", status_pass)    
    return f"Пропуск для {full_name} успешно добавлен и истечёт через час"

@tool(args_schema=ApartmentRequest)
def call_plumber(apartment_number: str, resident_name: str, time_of_visit: str) -> str:
    """Adds a request to call a plumber, including information about the apartment and the time of the visit."""
    request_id = f"{apartment_number}_{resident_name.lower().replace(' ', '_')}"

    plumbers_requests[request_id] = {
        "apartment_number":apartment_number,
        "resident_name":resident_name,
        "time_of_visit": time_of_visit
    }
    

    write_json_file('./json/plumbers_requests.json', plumbers_requests)
    return f"Запрос на вызов сантехника для квартиры {apartment_number} принят. Время визита: {time_of_visit}"



tools = [add_employee_pass, call_plumber]
llm = ChatMistralAI(api_key = API_KEY, model="mistral-large-latest", temperature=0)
llm_with_tools = llm.bind_tools(tools)


message = [
    ("system", "Отвечай только на русском языке. Ты являешься **ДОМОФОНОМ** и всегда общаешься уважительно и очень коротко, и отвечаешь только на вопросы человека про себя, или тот функцианал что у тебя есть"),
    ("system", "Человека зовут **{name}** и ему **{age}**"),
    ("human", "{message}")
]
promt = ChatPromptTemplate(message)


chain = promt | llm_with_tools




def chatbot(state: State) -> State:
    """Обрабатывает сообщения и обновляет состояние."""
    time.sleep(1)

    res = chain.invoke({
        "message": state['messages'],
        "name": state['name'],
        "age": state['age']
    })

    if "tool_calls" in res.additional_kwargs:
        return {
            "messages": res,
            "name": state['name'],
            "age": state['age']
        }

    return {
        "messages": {"role": "ai", "content": res.content},
        "name": state['name'],
        "age": state['age']
    }






graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[add_employee_pass, call_plumber])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [16]:

config = {"configurable": {"thread_id": "1"}}
user_input = "закажи мне пожалуйста сантехника в квартиру 1024 на 18:00"

events = graph.invoke(
    {"messages": [{"role": "user", "content": user_input}],
     "name": "Тестовый Тестович",
     "age": 20
     },
    config
)
# for event in events:
#     #.pretty_print()
#     a = event["messages"][-1]
events


{'messages': [HumanMessage(content='закажи мне пожалуйста сантехника в квартиру 1024 на 18:00', additional_kwargs={}, response_metadata={}, id='6fcaaea6-281b-42b3-95da-049e66c893db'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'psuaBn09y', 'function': {'name': 'call_plumber', 'arguments': '{"apartment_number": "1024", "resident_name": "Тестовый Тестович", "time_of_visit": "18:00"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 483, 'total_tokens': 540, 'completion_tokens': 57}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-8f7073d4-805a-43f7-9450-d2e2d8d8ee9e-0', tool_calls=[{'name': 'call_plumber', 'args': {'apartment_number': '1024', 'resident_name': 'Тестовый Тестович', 'time_of_visit': '18:00'}, 'id': 'psuaBn09y', 'type': 'tool_call'}], usage_metadata={'input_tokens': 483, 'output_tokens': 57, 'total_tokens': 540}),
  ToolMessage(content='Запрос на вызов сантехника для квартиры 1024 принят. Время визита: 18:00'

In [17]:

llm = ChatMistralAI(api_key = API_KEY, model="mistral-large-latest")
response = llm.stream("как у тебя дела?")
a = ''
for value in response:
    a += value.content
    print(a)

HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}